In [9]:
import numpy as np
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint
import pickle
from sklearn.preprocessing import StandardScaler
from keras.utils.vis_utils import plot_model

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, CuDNNLSTM, Multiply,Dropout
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
from keras.callbacks import TensorBoard

In [10]:
#address to getData->output folder
address = "C:\\Users\\djaym7\\Desktop\\Github\\data 02\\"

In [11]:
def loadData(id):
    with open(address + f'y_train_{id}','rb') as f:
        y_train=np.array(pickle.load(f))
    with open(address+f'y_test_{id}','rb') as f:
        y_test=np.array(pickle.load(f))
    with open(address+ f'X_train_{id}','rb') as f:
        X_train = np.array(pickle.load(f))
    with open(address+f'X_test_{id}','rb') as f:
        X_test = np.array(pickle.load(f))

    scalers = {}
    for k in range(X_train.shape[2]):
        scalers[k] = StandardScaler()
        X_train[:, k, :] = scalers[k].fit_transform(X_train[:, k, :]) 

    for j in range(X_test.shape[2]):
        X_test[:, j, :] = scalers[j].transform(X_test[:, j, :])
    
    print(f'id : {id}')
    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)

   
    return X_train,y_train,X_test,y_test


In [12]:
def plot_attention_map(model, input_dim, n_s = 96, num = 1, Tx =13 , Ty = 1):
    """
    Plot the attention map.
  
    """
    attention_map = np.zeros((Ty, Tx))
    Ty, Tx = attention_map.shape
    print(attention_map.shape)
    
    s0 = np.zeros((X_train.shape[0], n_s))
    c0 = np.zeros((X_train.shape[0], n_s))
    layer = model.layers[num]
    print('No layers','\t',layer)


    f = K.function(model.inputs, [layer.get_output_at(t) for t in range(Ty)])
    r = f([X_train, s0, c0])
    print(r)
    
    for t in range(Ty):
        for t_prime in range(Tx):
            attention_map[t][t_prime] = r[t][0,t_prime,0]
            
    print(attention_map)

    # Normalize attention map
#     row_max = attention_map.max(axis=1)
#     attention_map = attention_map / row_max[:, None]

    prediction = model.predict([X_train, s0, c0])
    
 
    
    # get the lengths of the string
    input_length = Tx
    output_length = Ty
    
    # Plot the attention_map
    plt.clf()
    f = plt.figure(figsize=(8, 8.5))
    ax = f.add_subplot(1, 1, 1)

    # add image
    i = ax.imshow(attention_map, interpolation='nearest', cmap='Blues')

    # add colorbar
    cbaxes = f.add_axes([0.2, 0, 0.6, 0.03])
    cbar = f.colorbar(i, cax=cbaxes, orientation='horizontal')
    cbar.ax.set_xlabel('Alpha value (Probability output of the "softmax")', labelpad=2)

    # add labels
    ax.set_yticks(range(output_length))

    ax.set_xticks(range(input_length))

    ax.set_xlabel('Input Sequence')
    ax.set_ylabel('Output Sequence')

    # add grid and legend
    ax.grid()

    #f.show()
    
    return attention_map
#plot_attention_map(model, input_dim, n_s = 96, num = 1, Tx =13 , Ty = 1)

In [18]:
total_num_people=[]
for i in range(1,92):
    if i <10:
        total_num_people.append('100'+str(i))
    else:
        total_num_people.append('10'+str(i))

for i in total_num_people:
    if True:
        X_train=[]
        X_test=[]
        y_train=[]
        y_test=[]
        X_train,y_train,X_test,y_test=loadData(i)
        
    else:
        continue


    #load data here
    #input_shape = batch_size,time_steps,input_dim

    #batch_size = X_train.shape[0]   #automatically taken no need to specify
    time_steps = X_train.shape[1]
    input_dim = X_train.shape[2]



    hid_pre_acts = [16]
    hid_post_acts = [96]
    #dense1s = [16,32,64,96,128,256]
    #dense2s = [16,32,64,96,128,256]
    for hid_pre_act in hid_pre_acts:
        for hid_post_act in hid_post_acts:
            # Defined shared layers as global variables
            repeator = RepeatVector(time_steps) #500
            concatenator = Concatenate(axis=-1)
            densor = Dense(1, activation = "relu")
            activator = Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
            dotor = Dot(axes = 1)

            #one step attention

            def one_step_attention(a,s_prev):

                s_prev = repeator(s_prev)
                concat = concatenator([a,s_prev])
                e = densor(concat)
                alphas = activator(e)
                K.print_tensor(alphas)
                context = dotor([alphas, a])

                return context
            
            post_activation_lstm_cell = CuDNNLSTM(hid_post_act,return_state=True)
            #output_layer = Dense(dense1,activation='relu')
            #onemore = Dense(dense2,activation='relu')
            fin_out = Dense(6,activation='softmax')

                # model

            def model(time_steps,input_dim,hid_post_act,hid_pre_act):

                X = Input(shape =(time_steps,input_dim))
                s0 = Input(shape=(hid_post_act,),name='s0')
                c0 = Input(shape=(hid_post_act,),name='c0')
                s=s0
                c=c0
                outputs=[]


                # Pre-LSTM 16
                a = CuDNNLSTM(hid_pre_act,return_sequences=True)(X)
                a = Dropout(0.4)(a)
                #6 is final output size
                for t in range(1):
                    context =one_step_attention(a,s)
                    s, _, c = post_activation_lstm_cell(context, initial_state=[s, c])
                    #out = output_layer(s)
                    #o = onemore(out)
                    k=fin_out(s)
                    outputs.append(k)


                model = Model([X, s0, c0], outputs)

                return model       

            model = model(time_steps,input_dim,hid_post_act,hid_pre_act)

            opt = Adam()
            #opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
            model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
            model.summary()
            plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
            print(X_train.shape)
            s0 = np.zeros((X_train.shape[0], hid_post_act))
            c0 = np.zeros((X_train.shape[0], hid_post_act))
            s0_t = np.zeros((X_test.shape[0], hid_post_act))
            c0_t = np.zeros((X_test.shape[0], hid_post_act))
            print(f'id : {i}')
            #filePath =f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\ExtendedSavedModels\\{i}.h5'
            #checkpoint = ModelCheckpoint(filepath=filePath,monitor='val_acc',mode='max',save_best_only=True,verbose=1,save_weights_only=False)
            model.fit([X_train, s0, c0], y_train, epochs=1,validation_data=([X_test,s0_t,c0_t],y_test)) #,callbacks=[checkpoint]
            #plot_attention_map(model, input_dim, n_s = 96, num = 1, Tx =13 , Ty = 1)

id : 1001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 500, 13)      0                                            
__________________________________________________________________________________________________
cu_dnnlstm_12 (CuDNNLSTM)       (None, 500, 16)      1984        input_6[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 96)           0                                            
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 500, 16)      0           cu_dnnlstm_12[0][0]              
__________________________________________________________________________________________________


KeyboardInterrupt: 

In [15]:
attention_values

NameError: name 'attention_values' is not defined